In [1]:
import json

calculator_tools_definition = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Perform basic arithmetic operations",
        "parameters": {
            "type": "object",
            "properties": {
                "operator": {
                    "type": "string",
                    "description": "Arithmetic operation to perform",
                    "enum": ["add", "subtract", "multiply", "divide"]
                },
                "first_number": {
                    "type": "number",
                    "description": "First number for calculation"
                },
                "second_number": {
                    "type": "number",
                    "description": "Second number for calculation"
                }
            },
            "required": ["operator", "first_number", "second_number"]
        }
    }
}

STEP 2: SET UP THE TOOL

In [2]:
def calculator(operator: str, first_number: float, second_number: float):
    if operator == 'add':
        return first_number + second_number
    elif operator == 'subtract':
        return first_number - second_number
    elif operator == 'multiply':
        return first_number * second_number
    elif operator == 'divide':
        if second_number == 0:
            raise ValueError("Cannot divide by zero")
            return first_number / second_number
        return None
    else:
        raise ValueError(f"Unsupported operator: {operator}")

In [3]:
from litellm import completion

tools = [calculator_tools_definition]

response_without_tool = completion(
    model="gpt-5-mini",
    messages=[{
        "role": "user",
        "content": "What is the capital of Pakistan ?",
        "tools": tools,
    }])

print(response_without_tool.choices[0].message.content)
print(response_without_tool.choices[0].message.tool_calls)

response_with_tool = completion(
    model='gpt-5-mini',
    messages=[{"role": "user", "content": "What is 1234 x 5678?"}],
    tools=tools
)
print(response_with_tool.choices[0].message.content)
print(response_with_tool.choices[0].message.tool_calls)

None
[ChatCompletionMessageToolCall(function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), id='call_tqKjpWXSLgKo3kDHrKLiw3ex', type='function')]


In [4]:
import json

ai_message = response_with_tool.choices[0].message

if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "calculator":
            result = calculator(**function_args)

STEP 5: FEEDING RESULTS BACK TO THE LLM

In [5]:
ai_message = response_with_tool.choices[0].message

messages = list()

messages.append({
    "role": "assistant",
    "content": ai_message.content,
    "tool_calls": ai_message.tool_calls,
})

if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "calculator":
            result = calculator(**function_args)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            })

final_response = completion(
    model="gpt-5-mini",
    messages=messages,
)

print("Messages:", messages)
print("Final answer:", final_response.choices[0].message.content)

Messages: [{'role': 'assistant', 'content': None, 'tool_calls': [{'function': {'arguments': '{"operator":"multiply","first_number":1234,"second_number":5678}', 'name': 'calculator'}, 'id': 'call_tqKjpWXSLgKo3kDHrKLiw3ex', 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': 'call_tqKjpWXSLgKo3kDHrKLiw3ex', 'content': '7006652'}]
Final answer: 1234 × 5678 = 7,006,652

Quick check:
- 1234×1000 = 1,234,000
- 1234×200 = 246,800
- 1234×30 = 37,020
- 1234×4 = 4,936
Sum = 1,234,000 + 246,800 + 37,020 + 4,936 = 7,006,656 — wait, that sums incorrectly.

Let me recompute cleanly:
1234×5678 = 1234×(5000+600+70+8)
- 1234×5000 = 6,170,000
- 1234×600 = 740,400
- 1234×70 = 86,380
- 1234×8 = 9,872
Sum = 6,170,000 + 740,400 + 86,380 + 9,872 = 7,006,652

So the correct product is 7,006,652.


Basic web search function using Tavily

In [6]:
import os
from tavily import TavilyClient
from dotenv import load_dotenv

load_dotenv()

tavily_client = TavilyClient(os.getenv('TAVILY_API_KEY'))


def search_web(query: str, max_results: int = 2) -> list:
    response = tavily_client.search(query=query, max_results=max_results)
    return response.get("results")

Testing the web search function

In [7]:
search_web("Kipchoge's marathon world record")

[{'url': 'https://www.menshealth.com/fitness/a29447891/eliud-kipchoge-sub-2-minute-marathon-not-official-world-record/',
  'title': "Why Eliud Kipchoge's 1:59 Marathon Won't Be an Official World ...",
  'content': 'Kipchoge already holds the official marathon world record, having completed the 2018 Berlin marathon in 2:01:39. In 2016, he won the gold medal',
  'score': 0.8793233,
  'raw_content': None},
 {'url': 'https://www.worldmarathonmajors.com/content-hub/kipchoge-shatters-world-record',
  'title': 'Kipchoge shatters world record - Abbott World Marathon Majors',
  'content': 'If he wasn\'t already the greatest, Eliud Kipchoge made it officialat the 2018 BMW BERLIN-MARATHON as he smashed the world record with a time of 2:01:39. The Kenyan, already a three-time AbbottWMM Series champion and defending Berlin champion, set out with three pacemakers at an unrelenting tempo, passing through halfway in 61:06, just a slither outside his planned target of 61:00. Wilson Kipsang, who set his

Web search function with additional options

In [8]:
def search_web(
        query: str,
        max_results: int = 2,
        topic: str = "general",
        time_range: str | None = None,
        country: str | None = None,
) -> list:
    response = tavily_client.search(
        query=query,
        max_results=max_results,
        topic="general",
        time_range=time_range,
        country=country
    )

    return response.get("results")

Web search function with error handling

In [9]:
def search_web(
        query: str,
        max_results: int = 5,
        topic: str = "general",
        time_range: str | None = None,
) -> list | str:
    try:
        response = tavily_client.search(
            query,
            max_results=max_results,
            topic=topic,
            time_range=time_range,
        )

        return response.get("results")
    except Exception as e:
        return f"Error: Search failed: {e}"

Extract tool metadata using inspect

In [10]:
import inspect


def example_tool(input_1: str, input_2: int = 1):
    """doc string for example_tool"""
    return


print(f"function name:{example_tool.__name__}")
print(f"function docstring:{example_tool.__doc__}")
print(f"function signature: {inspect.signature(example_tool)}")

function name:example_tool
function docstring:doc string for example_tool
function signature: (input_1: str, input_2: int = 1)


Function to tool schema converter

In [11]:
def function_to_input_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}

    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name for param in signature.parameters.values()

        if param.default == inspect._empty
    ]

    return {
        "type": "object",
        "properties": parameters,
        "required": required,
    }

Function to tool schema converter

In [12]:
def format_tool_definition(name: str, description: str, parameters: dict) -> dict:
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": parameters,
        }
    }


def function_to_tool_definition(func) -> dict:
    return format_tool_definition(
        func.__name__,
        func.__doc__ or "",
        function_to_input_schema(func),
    )


In [13]:
search_tool_definition = function_to_tool_definition(search_web)
print(search_tool_definition)

{'type': 'function', 'function': {'name': 'search_web', 'description': '', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}, 'max_results': {'type': 'integer'}, 'topic': {'type': 'string'}, 'time_range': {'type': 'string'}}, 'required': ['query']}}}


Tool execution utility function

In [14]:
def tool_execution(tool_box, tool_call):
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    tool_result = tool_box[function_name](**function_args)

    return tool_result

Simple agent loop for tool execution

In [15]:
def simple_agent_loop(system_prompt, question):
    tools = [search_web]
    tool_box = {tool.__name__: tool for tool in tools}
    tool_definitions = [function_to_tool_definition(tool) for tool in tools]

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    while True:
        response = completion(
            model="gpt-5-mini",
            messages=messages,
            tools=tool_definitions
        )
        assistant_message = response.choices[0].message

        if assistant_message.tool_calls:
            messages.append(assistant_message)
            for tool_call in assistant_message.tool_calls:
                tool_result = tool_execution(tool_box, tool_call)
                messages.append({
                    "role": "tool",
                    "content": str(tool_result),
                    "tool_call_id": tool_call.id
                })
            else:
                return assistant_message.content

Testing the agent loop

In [16]:
system_prompt = """You are a helpful assistant.
Use the search tool when you need current information."""
result = simple_agent_loop(
    system_prompt,
    "Who won the 2025 Nobel Prize in Physics?"
)
print(result)

None


In [17]:
import asyncio
import os
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


In [18]:
server_params = StdioServerParameters(
    command="npx",
    args=["-y", "tavily-mcp@latest"],
    env={
        "TAVILY_API_KEY": os.getenv("TAVILY_API_KEY"),
    }
)

async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()
        # List available tools
        tools_result = await session.list_tools()
        print("Available tools:")
        for tool in tools_result.tools:
            print(f" - {tool.name}: {tool.description[:60]}...")
            result = await session.call_tool(
                "tavily_search",
                arguments={"query": "2025 Nobel Physics"}
            )
            print("Search Result:")
            print(result.content)

Search Result:
[TextContent(type='text', text='Detailed Results:\n\nTitle: Nobel Prize in Physics 2025 - NobelPrize.org\nURL: https://www.nobelprize.org/prizes/physics/2025/summary/\nContent: Nobel Prize in Physics 2025 · John Clarke · Michel H. Devoret · John M. Martinis. Prize share: 1/3. The Nobel Prize in Physics 2025 was awarded jointly\n\nTitle: John Clarke, UC Berkeley emeritus professor, awarded 2025 Nobel ...\nURL: https://ls.berkeley.edu/news/john-clarke-uc-berkeley-emeritus-professor-awarded-2025-nobel-prize-physics\nContent: # John Clarke, UC Berkeley emeritus professor, awarded 2025 Nobel Prize in Physics. John Clarke, an emeritus professor of physics at the University of California, Berkeley, was awarded the 2025 Nobel Prize in Physics for his work on quantum tunneling, one of many strange aspects of quantum mechanics. The\xa0Nobel Prize committee\xa0honored the three “for the discovery of macroscopic quantum mechanical tunneling and energy quantization in an electric cir

In [20]:
def mcp_tools_to_openai_format(mcp_tools) -> list[dict]:
    """Convert MCP tools to OPEN AI format"""

    return [
        format_tool_definition(
            name=tool.name,
            description=tool.description,
            parameters=tool.inputSchema,
        )
        for tool in mcp_tools.tools
    ]

Retrieving tools in OpenAI format

In [21]:
async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()

        # List available tools
        tool_result = await session.list_tools()

        open_ai_format_tools = mcp_tools_to_openai_format(tool_result)

        for tool in open_ai_format_tools:
            print(tool)

{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'Search the web for current information on any topic. Use for news, facts, or data beyond your knowledge cutoff. Returns snippets and source URLs.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query'}, 'search_depth': {'type': 'string', 'enum': ['basic', 'advanced', 'fast', 'ultra-fast'], 'description': "The depth of the search. 'basic' for generic results, 'advanced' for more thorough search, 'fast' for optimized low latency with high relevance, 'ultra-fast' for prioritizing latency above all else", 'default': 'basic'}, 'topic': {'type': 'string', 'enum': ['general'], 'description': 'The category of the search. This will determine which of our agents will be used for the search', 'default': 'general'}, 'time_range': {'type': 'string', 'description': "The time range back from the current date to include in the search results. This feature is available for bot

Building MCP Server

In [22]:
from tavily import TavilyClient
from mcp.server.fastmcp import FastMCP

In [30]:
server_params = StdioServerParameters(
    command="uv",
    args=["run", "custom_tavily_mcp.py"],
    env={
        "TAVILY_API_KEY": os.getenv("TAVILY_API_KEY"),
    }
)

async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()

        # List available tools
        tool_result = await session.list_tools()

        print("Available tools:")
        for tool in tool_result.tools:
            print(f" - {tool.name} : {tool.description}")

Available tools:
 - search_web : 
    Search the web using Tavily API
    Args:
        query: Search query string
        max_results: Maximum number of results to return(default: 5)
    Returns:
          Search results in formatted string
    
